In [1]:
import os
from diffusers import DiffusionPipeline
import os
from typing import List
from cog import BasePredictor, Input, Path
import torch
from diffusers import (
    StableDiffusionPipeline,
    StableDiffusionImg2ImgPipeline,
    PNDMScheduler,
    LMSDiscreteScheduler,
    DDIMScheduler,
    EulerDiscreteScheduler,
    EulerAncestralDiscreteScheduler,
    DPMSolverMultistepScheduler,
)
from PIL import Image


EXTERNAL = False

# if download
model_id = "stabilityai/stable-diffusion-2-1"
# os.makedirs(MODEL_CACHE, exist_ok=True)
#local storage
model_id = "./diffusers-cache/models--stabilityai--stable-diffusion-2-1/snapshots/36a01dc742066de2e8c91e7cf0b8f6b53ef53da1/"

MODEL_CACHE = "diffusers-cache"
scheduler='DPMSolverMultistep'


# download heigths
#pipe = DiffusionPipeline.from_pretrained(
#    model_id,
#    cache_dir=MODEL_CACHE,
#)

if EXTERNAL:
  %store -r WIDTH
  %store -r HEIGHT
  %store -r TEXT
  %store -r VISUALIZE
  %store -r SAVE_FOLDER
  %store -r PROJECT_NAME
  %store -r SHOW_OUTPUT
  %store -r IMG_NUMBER
  %store -r THUMBNAIL
  %store -r IMG_NUMBER_IMG2IMG

else:
  WIDTH = 768
  HEIGHT = 768
  VISUALIZE = True  
  SAVE_FOLDER = "D:\\Deletar\\p_gen"
  PROJECT_NAME = "THE FELLOWSHIP OF THE RING"
  STYLES_FOLDER = ".\\styles"
  SHOW_OUTPUT = True
  IMG_NUMBER = 2
  IMG_NUMBER_IMG2IMG = 5


txt2img_pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            cache_dir=MODEL_CACHE,
            local_files_only=True,
        ).to("cuda")    

txt2img_pipe.set_use_memory_efficient_attention_xformers(True)

img2img_pipe = StableDiffusionImg2ImgPipeline(
            vae=txt2img_pipe.vae,
            text_encoder=txt2img_pipe.text_encoder,
            tokenizer=txt2img_pipe.tokenizer,
            unet=txt2img_pipe.unet,
            scheduler=txt2img_pipe.scheduler,
            safety_checker=txt2img_pipe.safety_checker,
            feature_extractor=txt2img_pipe.feature_extractor,
        ).to("cuda")

img2img_pipe.set_use_memory_efficient_attention_xformers(True)
pipe = img2img_pipe
project_folder = f"{SAVE_FOLDER}\\{PROJECT_NAME}"



A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [2]:
def make_scheduler(name, config):
    return {
        "PNDM": PNDMScheduler.from_config(config),
        "KLMS": LMSDiscreteScheduler.from_config(config),
        "DDIM": DDIMScheduler.from_config(config),
        "K_EULER": EulerDiscreteScheduler.from_config(config),
        "K_EULER_ANCESTRAL": EulerAncestralDiscreteScheduler.from_config(config),
        "DPMSolverMultistep": DPMSolverMultistepScheduler.from_config(config),
    }[name]
    
pipe.scheduler = make_scheduler(scheduler, pipe.scheduler.config)

""" MODEL_ID = "stabilityai/stable-diffusion-2-1"
MODEL_CACHE = "diffusers-cache"

MODEL_ID = "./diffusers-cache/models--stabilityai--stable-diffusion-2-1/snapshots/36a01dc742066de2e8c91e7cf0b8f6b53ef53da1/" """

class Predictor(BasePredictor):
    def setup(self):       
        """ print("Loading pipeline...")
        self.txt2img_pipe = StableDiffusionPipeline.from_pretrained(
            MODEL_ID,
            cache_dir=MODEL_CACHE,
            local_files_only=True,
        ).to("cuda")
        self.img2img_pipe = StableDiffusionImg2ImgPipeline(
            vae=self.txt2img_pipe.vae,
            text_encoder=self.txt2img_pipe.text_encoder,
            tokenizer=self.txt2img_pipe.tokenizer,
            unet=self.txt2img_pipe.unet,
            scheduler=self.txt2img_pipe.scheduler,
            safety_checker=self.txt2img_pipe.safety_checker,
            feature_extractor=self.txt2img_pipe.feature_extractor,
        ).to("cuda") """

    @torch.inference_mode()
    def predict(
        self,
        prompt: str = Input(
            description="Input prompt",
            default="A fantasy landscape, trending on artstation",
        ),
        negative_prompt: str = Input(
            description="The prompt NOT to guide the image generation. Ignored when not using guidance",
            default=None,
        ),
        image: Path = Input(
            description="Inital image to generate variations of.",
        ),
        width: int = Input(
            description="Width of output image. Maximum size is 1024x768 or 768x1024 because of memory limits",
            choices=[128, 256, 384, 448, 512, 576, 640, 704, 768, 832, 896, 960, 1024],
            default=512,
        ),
        height: int = Input(
            description="Height of output image. Maximum size is 1024x768 or 768x1024 because of memory limits",
            choices=[128, 256, 384, 448, 512, 576, 640, 704, 768, 832, 896, 960, 1024],
            default=512,
        ),
        prompt_strength: float = Input(
            description="Prompt strength when providing the image. 1.0 corresponds to full destruction of information in init image",
            default=0.8,
        ),
        num_outputs: int = Input(
            description="Number of images to output. Higher number of outputs may OOM.",
            ge=1,
            le=8,
            default=1,
        ),
        num_inference_steps: int = Input(
            description="Number of denoising steps", ge=1, le=500, default=25
        ),
        guidance_scale: float = Input(
            description="Scale for classifier-free guidance", ge=1, le=20, default=7.5
        ),
        scheduler: str = Input(
            default="DPMSolverMultistep",
            choices=["DDIM", "K_EULER", "DPMSolverMultistep", "K_EULER_ANCESTRAL", "PNDM", "KLMS"],
            description="Choose a scheduler.",
        ),
        seed: int = Input(
            description="Random seed. Leave blank to randomize the seed", default=None
        ),
        output_location: str = Input(
            description="Output location", default="./"),
        index: int = Input(
            description="Index", default="0"),
    ) -> List[Path]:
       
        if seed is None:
            seed = int.from_bytes(os.urandom(2), "big")
        # print(f"Using seed: {seed}")

        
        extra_kwargs = {
            "image": Image.open(image).convert("RGB"),
            "strength": prompt_strength,
        }
        

        generator = torch.Generator("cuda").manual_seed(seed)
        output = pipe(
            # prompt=[prompt] * num_outputs if prompt is not None else None,
            prompt= prompt,
            guidance_scale=guidance_scale,
            generator=generator,
            num_inference_steps=num_inference_steps,
            **extra_kwargs,
        )

        output_paths = []
        for i, sample in enumerate(output.images):
            output_path = f"{output_location}\\result_a{index}.jpg"
            sample.save(output_path)
            # output_paths.append(Path(output_path))

        return output_paths




run_scr = Predictor()

In [5]:
""" for i in range(5):
    run_scr.predict(prompt = [""], image="./c.jpg", seed = int.from_bytes(os.urandom(2), "big"), output_location ="./" , index = i,
    prompt_strength= 0.3, guidance_scale= 10.5, num_inference_steps=15) """
# str = 0.7 and scale = 2

if EXTERNAL:
    cont = 0
    if EXTERNAL:
        if (not os.path.exists(f"{project_folder}\\style\\")):
            os.makedirs(f"{project_folder}\\style\\")
        for folder_name in os.listdir(os.path.join(f"{project_folder}\\img\\")):
            print(folder_name)
            for filename in os.listdir(os.path.join(f"{project_folder}\\img\\{folder_name}")):
                if (not os.path.exists(f"{project_folder}\\style\\{folder_name}")):
                        os.makedirs(f"{project_folder}\\style\\{folder_name}")
                if (filename == "c.jpg"):
                    for i in range(IMG_NUMBER_IMG2IMG):
                        run_scr.predict(prompt = "", 
                            image=f"{project_folder}\\img\\{folder_name}\\{filename}", seed = int.from_bytes(os.urandom(2), "big"),
                            output_location =f"{project_folder}\\style\\{folder_name}" , index = i,
                            prompt_strength= 0.4, guidance_scale = 4, num_inference_steps=15
                        )
else:
    for i in range(100):
        run_scr.predict(prompt = "(((beautiful))), detailed, (((young woman))), high definition, realistic", 
        image="./c.jpg", seed = int.from_bytes(os.urandom(2), "big"), output_location ="./random" , index = i,
        prompt_strength= 0.6, guidance_scale = 7.5, num_inference_steps=15)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

KeyboardInterrupt: 